<a href="https://colab.research.google.com/github/kaique-vitiello/Python/blob/master/Atividade_Aula_6_ELT_com_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1 - Configurando o Spark no Python**

Etapa responsável por realizar o Download e configuração do Apache Spark.

In [1]:
#Instacao e configuracao do Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Download do Apache Spark
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#Descompatacao do Apache Spark
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#Instalacao do Findspark
!pip install -q findspark

**2 - Configurando o Ambiente**

Etapa responsável por configurar o ambiente do Spark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

**3 - Configurando a sessão do Spark**

Etapa responsável por inicializar a sessão do Spark

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

**4 - Validando a Configuração e Versão do Spark**

Etapa de verificação 

In [4]:
import pyspark
print(pyspark.__version__)

3.0.1


**5 - Download dos Dados do Portal da Transparência**

Etapa responsável por realizar o Download das informações de Gastos por meio de cartão de pagamento providas através da API: http://www.portaltransparencia.gov.br/api-de-dados/cartoes?pagina=1

In [8]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados: 06e7683b347936e84902f24cd6a798b2' 'http://www.portaltransparencia.gov.br/api-de-dados/viagens?dataIdaDe=01%2F07%2F2020&dataIdaAte=01%2F08%2F2020&dataRetornoDe=01%2F07%2F2020&dataRetornoAte=01%2F08%2F2020&codigoOrgao=20000&pagina=1' > consulta_portal_transparencia.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26647    0 26647    0     0  11864      0 --:--:--  0:00:02 --:--:-- 11864


**6 - Carregando os dados no Spark**

Etapa paga a carga de dados no Spark.

In [9]:
#Instancia o contexto do Spark
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Governo Federal sobre as Viagens
path = "/content/consulta_portal_transparencia.json"
#Declara dataframe df com o conteudo da api que estava no JSON
df = spark.read.json(path)

**7 - Explorando os dados que foram obtidos**

Etapa de exploração dos dados.

In [10]:
#Imprime a estrutura de dados do nosso dataframe
df.printSchema()
#Imprime uma amosta de dados ( para especificar as linhas basta colocar a quantidade entre ())
df.show()

root
 |-- beneficiario: struct (nullable = true)
 |    |-- cpf: string (nullable = true)
 |    |-- cpfFormatado: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- nomeSemAcento: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- dataFimAfastamento: string (nullable = true)
 |-- dataInicioAfastamento: string (nullable = true)
 |-- dimViagem: struct (nullable = true)
 |    |-- ano: long (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- justificativaUrgente: string (nullable = true)
 |    |-- motivo: string (nullable = true)
 |    |-- numPcdp: string (nullable = true)
 |    |-- pcdp: string (nullable = true)
 |    |-- viagemUrgente: boolean (nullable = true)
 |-- funcao: string (nullable = true)
 |-- id: long (nullable = true)
 |-- orgao: struct (nullable = true)
 |    |-- cnpj: string (nullable = true)
 |    |-- codigoSIAFI: string (nullable = true)
 |    |-- descricaoPoder: string (nullable = t

**8 - Tratando e transformando os dados**

In [24]:
dfPrepared = df.selectExpr("beneficiario.nome","dimViagem.ano","dimViagem.motivo","pessoa.municipio.nomeIBGE as municipio", "tipoViagem.descricao", "valorTotalViagem")
dfPrepared.show()

+--------------------+----+--------------------+---------------+---------+----------------+
|                nome| ano|              motivo|      municipio|descricao|valorTotalViagem|
+--------------------+----+--------------------+---------------+---------+----------------+
|LUIZ HENRIQUE GOM...|2020|OFÍCIO Nº 500/202...| RIO DE JANEIRO| Nacional|         1690.03|
|PAULO RICARDO DA ...|2020|OFÍCIO Nº 500/202...|      NILÓPOLIS| Nacional|         1605.03|
|LUCAS PINTO DA SILVA|2020|OFÍCIO Nº 500/202...|DUQUE DE CAXIAS| Nacional|         1605.03|
|GERSON RICARDO PA...|2020|SOLICITAÇÃO DE DI...|           IJUÍ| Nacional|          321.28|
|PAULO RICARDO PEI...|2020|SOLICITAÇÃO DE DI...|       BRASÍLIA| Nacional|          321.28|
|  DENIS BENINI SOUSA|2020|SOLICITAÇÃO DE DI...|       BRASÍLIA| Nacional|          321.28|
|FLAVIO FRANCISCO ...|2020|SOLICITAÇÃO DE DI...|       BRASÍLIA| Nacional|          321.28|
|ROSSANO AQUINO BE...|2020|SOLICITAÇÃO DE DI...|       BRASÍLIA| Nacional|      

**9 - Analise das informações e resultados obtidos**

Etapa onde são realizadas as analises utilizando linguagem SQL para por exemplo descobrir quem mais gastou no ano vigente.

In [25]:
#Declara tabela temporaria no SparkSQL
dfPrepared.createOrReplaceTempView("dfPrepared")

In [34]:
#avalia o total gasto com viagem
topTravels = spark.sql("select sum(valorTotalViagem) as valor_total from dfPrepared")
topTravels.show()

+-----------+
|valor_total|
+-----------+
|    8356.89|
+-----------+

